In [1]:
import pandas as pd
import numpy as np
import os
import pandas_gbq
from google.cloud import bigquery
import glob
import openpyxl
import re

In [2]:
os.chdir('G:\\\\Drives compartilhados\\\\República.org\\\\4. Equipes\\\\Dados e Comunicação\\\\DADOS\\\\415 - Repositório de Dados\\\\Repositório Local')


Chamando as bases

In [3]:
df21 = pd.read_excel('MUNIC//Base_MUNIC_2021.xlsx', sheet_name='Educação', usecols=['CodMun','Medu161a','MEDU161b'])
df21

,CodMun,Medu161a,MEDU161b
0,1100015,-,-
1,1100023,954,2002
2,1100031,616,2009
3,1100049,2736,2010
4,1100056,1947,2011
...,...,...,...
5565,5222005,879,2008
5566,5222054,574,2011
5567,5222203,222,2009
5568,5222302,414,2009


In [4]:
uf = pd.read_excel('MUNIC//Base_MUNIC_2021.xlsx', sheet_name = 'Variáveis externas', usecols=[0,1,2,3])
uf 

,CodMun,UF,Cod UF,Mun
0,1100015,RO,11,Alta Floresta DOeste
1,1100023,RO,11,Ariquemes
2,1100031,RO,11,Cabixi
3,1100049,RO,11,Cacoal
4,1100056,RO,11,Cerejeiras
...,...,...,...,...
5565,5222005,GO,52,Vianópolis
5566,5222054,GO,52,Vicentinópolis
5567,5222203,GO,52,Vila Boa
5568,5222302,GO,52,Vila Propício


Renomeando as colunas

In [5]:
df21= df21.rename(columns={'CodMun':'id_municipio',
                           'Medu161a':'lei_criacao_plano_carreira',
                           'MEDU161b':'ano_criacao_lei'}) 

In [6]:
##adicionando a coluna de 'ano' em cada um dos dfs
df21['ano']=2021 

In [7]:
x= uf.pivot_table(columns=('CodMun', 'UF','Cod UF', 'Mun'), aggfunc='size')


In [8]:
uf = pd.DataFrame(x).reset_index()[['CodMun', 'UF','Cod UF', 'Mun']]

In [9]:
df = df21.merge(uf, right_on='CodMun',left_on='id_municipio') #adicionando siglas e nome das UFs e dos municipios


In [10]:
df = df.drop(['CodMun'], axis=1) #eliminando coluna repetida

In [11]:
df= df.rename(columns={'UF':'sigla_uf',
                'Cod UF':'cod_uf',
                'Mun':'nome_municipio'}) #padronizando

In [13]:
df.columns

Index(['id_municipio', 'lei_criacao_plano_carreira', 'ano_criacao_lei', 'ano',
       'sigla_uf', 'cod_uf', 'nome_municipio'],
      dtype='object')

In [14]:
#padronizando dados das colunas
df['lei_criacao_plano_carreira']=np.where(df['lei_criacao_plano_carreira']=='-','Sem dados',df['lei_criacao_plano_carreira'])
df['lei_criacao_plano_carreira']=np.where(df['lei_criacao_plano_carreira']=='(*) Não soube informar','Sem dados',df['lei_criacao_plano_carreira'])

df['ano_criacao_lei']=np.where(df['ano_criacao_lei']=='(*) Não soube informar',np.nan,df['ano_criacao_lei'])
df['ano_criacao_lei']=np.where(df['ano_criacao_lei']=='Não informou',np.nan,df['ano_criacao_lei'])
df['ano_criacao_lei']=np.where(df['ano_criacao_lei']=='-',np.nan,df['ano_criacao_lei'])
df['ano_criacao_lei']=np.where(df['ano_criacao_lei']=='Recusa',np.nan,df['ano_criacao_lei'])


In [44]:
df['ano_criacao_lei'].unique()

array([  nan, 2002., 2009., 2010., 2011., 2008., 2013., 2019., 2007.,
       2001., 2012., 2015., 2005., 2017., 2004., 2014., 2020., 2016.,
       1999., 2021., 1998., 2006., 2018., 2000., 2003., 1991., 1997.,
       1990., 1994., 1996., 1986., 1995., 1988., 1993., 1989., 1992.,
       1987., 1983., 1976.])

In [49]:
limites = [0,1980,1990,2000, 2010, 2020, 9999] #criando uma nova coluna (periodo_anual) com base na coluna 'ano_criacao_lei'
categorias = ['Década de 70','Década de 80','Década de 90', 'Década de 2000', "Década de 2010", "Década de 2020" ]

df['periodo_anual'] = pd.cut(df['ano_criacao_lei'], bins=limites, labels=categorias)

C:\Users\ana.sales_republica\AppData\Local\Temp\ipykernel_15804\2967302833.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['periodo_anual'] = pd.cut(df['ano_criacao_lei'], bins=limites, labels=categorias)


In [39]:
df['ano_criacao_lei'] = df['ano_criacao_lei'].astype('float') ##transformando em float para obter categoria

C:\Users\ana.sales_republica\AppData\Local\Temp\ipykernel_15804\2865004523.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ano_criacao_lei'] = df['ano_criacao_lei'].astype('float') ##transformando em float para obter categoria


In [29]:
a = df[df['periodo_anual']=='Década de 2000']
a

,ano,sigla_uf,id_municipio,nome_municipio,ano_criacao_lei,lei_criacao_plano_carreira,periodo_anual
1,2021,RO,1100023,Ariquemes,2002.0,954,Década de 2000
2,2021,RO,1100031,Cabixi,2009.0,616,Década de 2000
3,2021,RO,1100049,Cacoal,2010.0,2736,Década de 2000
5,2021,RO,1100064,Colorado do Oeste,2008.0,40,Década de 2000
10,2021,RO,1100114,Jaru,2007.0,1036,Década de 2000
...,...,...,...,...,...,...,...
5558,2021,GO,5221551,Turvelândia,2007.0,156,Década de 2000
5561,2021,GO,5221700,Uruana,2004.0,934,Década de 2000
5565,2021,GO,5222005,Vianópolis,2008.0,879,Década de 2000
5567,2021,GO,5222203,Vila Boa,2009.0,222,Década de 2000


In [28]:
df = df[['ano','sigla_uf','id_municipio','nome_municipio','ano_criacao_lei','lei_criacao_plano_carreira', 'periodo_anual']] #organizando as colunas


Subindo para o GBQ

In [30]:
client = bigquery.Client()
dataset_ref = client.dataset('legislacoes')

In [66]:
df.columns

Index(['ano', 'sigla_uf', 'id_municipio', 'nome_municipio', 'ano_criacao_lei',
       'lei_criacao_plano_carreira', 'periodo_anual'],
      dtype='object')

In [79]:
df1 = df[['ano', 'sigla_uf', 'id_municipio', 'nome_municipio','ano_criacao_lei',
       'lei_criacao_plano_carreira','periodo_anual']]

In [80]:
schema=[bigquery.SchemaField('ano','INTEGER',description='Ano da apuração daquele dado'),
        bigquery.SchemaField('sigla_uf','STRING',description='Sigla da UF'),
        bigquery.SchemaField('id_municipio','INTEGER',description='Identificador do município pelo IBGE'),
        bigquery.SchemaField('nome_municipio','STRING',description='Nome do município da observação'),       
       bigquery.SchemaField('ano_criacao_lei','Float',description='Ano de criação da lei em observação'),
        #bigquery.SchemaField('lei_criacao_plano_carreira','STRING',description='Ano em que a lei de criação de plano de carreira foi criada'),
       bigquery.SchemaField('periodo_anual','STRING',description='Período anual referente a observação')
        
        ]

In [76]:
df1.head()

,ano,sigla_uf,id_municipio,nome_municipio,lei_criacao_plano_carreira
0,2021,RO,1100015,Alta Floresta DOeste,Sem dados
1,2021,RO,1100023,Ariquemes,954
2,2021,RO,1100031,Cabixi,616
3,2021,RO,1100049,Cacoal,2736
4,2021,RO,1100056,Cerejeiras,1947


In [50]:
df.dtypes

ano                              int64
sigla_uf                        object
id_municipio                     int64
nome_municipio                  object
ano_criacao_lei                float64
lei_criacao_plano_carreira      object
periodo_anual                 category
dtype: object

In [81]:
table_ref = dataset_ref.table('MUNIC_lei_plano_carreira_por_ano')
job_config = bigquery.LoadJobConfig(schema=schema)
job = client.load_table_from_dataframe(df1,table_ref, job_config=job_config)
job.result() 

ArrowTypeError: Expected bytes, got a 'int' object